<a href="https://colab.research.google.com/github/ValentinCord/HandsOnAI_2/blob/main/NLP_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!/opt/bin/nvidia-smi
!rm -rf sample_data

Wed Dec  7 11:08:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    29W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip3 install transformers
!pip3 install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.2 MB/s 
     |████████████████████████████████| 182 kB 59.4 MB/s 
     |████████████████████████████████| 7.6 MB 49.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 5.1 MB/s 
     |████████████████████████████████| 132 kB 67.1 MB/s 
     |████████████████████████████████| 212 kB 69.5 MB/s 
     |████████████████████████████████| 127 kB 74.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
# basics 
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

# transformers 
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer
from transformers import CamembertModel, CamembertTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# plot 
import matplotlib.pyplot as plt 
import seaborn as sns 

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Lecture et analyse pandas

In [5]:
train_path = '/content/drive/MyDrive/HandOnAI_2_NLP/fake_train.csv'
test_path = '/content/drive/MyDrive/HandOnAI_2_NLP/fake_test.csv'

In [6]:
df = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

In [7]:
df = df.drop(['Unnamed: 0', 'target_name'], axis = 1)
df_test = df_test.drop(['Unnamed: 0', 'target_name'], axis = 1)

In [8]:
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('french'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    #text = REPLACE_BY_SPACE_RE.sub(' ', text)
    #text = BAD_SYMBOLS_RE.sub('', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
df['data'] = df['data'].apply(clean_text)
df['data'] = df['data'].str.replace('\d+', '')

<ipython-input-9-eb4163588f94>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['data'] = df['data'].str.replace('\d+', '')


In [10]:
df.head()

,data,label
0,e jour consécutif grève reconductible contre r...,0
1,"depuis plusieurs mois, initiatives chercheurs ...",0
2,google vient d'introduire mise jour applicatio...,0
3,portrait. chacun s’empresse autour d’elle tand...,0
4,"« n’y a risque pénurie » carburant, a déclaré ...",0


In [11]:
def get_split(text1):
    l_total = []
    l_parcial = []
    if len(text1.split())//150 >0:
        n = len(text1.split())//150
    else: 
        n = 1
    for w in range(n):
        if w == 0:
            l_parcial = text1.split()[:200]
            l_total.append(" ".join(l_parcial))
        else:
            l_parcial = text1.split()[w*150:w*150 + 200]
            l_total.append(" ".join(l_parcial))
    return l_total

In [12]:
df['text_split'] = df['data'].apply(get_split)
df.head()

,data,label,text_split
0,e jour consécutif grève reconductible contre r...,0,[e jour consécutif grève reconductible contre ...
1,"depuis plusieurs mois, initiatives chercheurs ...",0,"[depuis plusieurs mois, initiatives chercheurs..."
2,google vient d'introduire mise jour applicatio...,0,[google vient d'introduire mise jour applicati...
3,portrait. chacun s’empresse autour d’elle tand...,0,[portrait. chacun s’empresse autour d’elle tan...
4,"« n’y a risque pénurie » carburant, a déclaré ...",0,"[« n’y a risque pénurie » carburant, a déclaré..."


In [13]:
train_l = []
label_l = []
index_l =[]
for idx,row in df.iterrows():
    for l in row['text_split']:
        train_l.append(l)
        label_l.append(row['label'])
        index_l.append(idx)
len(train_l), len(label_l), len(index_l)

(2606, 2606, 2606)

In [14]:
df = pd.DataFrame({'data':train_l, 'label':label_l})
df.head()

,data,label
0,e jour consécutif grève reconductible contre r...,0
1,"depuis plusieurs mois, initiatives chercheurs ...",0
2,google vient d'introduire mise jour applicatio...,0
3,portrait. chacun s’empresse autour d’elle tand...,0
4,"beaucoup. » car deirdre mccloskey – cas rare, ...",0


In [15]:
new_df=df

df_target=pd.DataFrame(new_df['label'])
df_target['temp']=0
for i in range(len(df_target)):
    if df_target['label'].loc[i]==0:
        df_target['temp'].loc[i]=1
new_df1=new_df.drop(['label'],axis=1)
new_df2=pd.DataFrame(df_target)

new_df3=pd.concat([new_df1,new_df2],axis = 1)
df=new_df3

df['label'] = df[df.columns[1:]].values.tolist()
new_df = df[['data', 'label']].copy()
df = new_df
df = df.rename(columns={'data': 'text'})
df.head()

,text,label
0,e jour consécutif grève reconductible contre r...,"[0, 1]"
1,"depuis plusieurs mois, initiatives chercheurs ...","[0, 1]"
2,google vient d'introduire mise jour applicatio...,"[0, 1]"
3,portrait. chacun s’empresse autour d’elle tand...,"[0, 1]"
4,"beaucoup. » car deirdre mccloskey – cas rare, ...","[0, 1]"


In [16]:
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [17]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 1
VALID_BATCH_SIZE = 1
EPOCHS = 1
LEARNING_RATE = 1e-05

In [21]:
!pip install sentencepiece
tokenizer = AutoTokenizer.from_pretrained("cmarkea/distilcamembert-base")
camembert = CamembertModel.from_pretrained("cmarkea/distilcamembert-base")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Downloading:   0%|          | 0.00/236 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/732 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/811k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/273M [00:00<?, ?B/s]

Some weights of the model checkpoint at cmarkea/distilcamembert-base were not used when initializing CamembertModel: ['lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertModel were not initialized from the model checkpoint at cmarkea/distilcamembert-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream

In [22]:
df.head()

,text,label
0,e jour consécutif grève reconductible contre r...,"[0, 1]"
1,"depuis plusieurs mois, initiatives chercheurs ...","[0, 1]"
2,google vient d'introduire mise jour applicatio...,"[0, 1]"
3,portrait. chacun s’empresse autour d’elle tand...,"[0, 1]"
4,"beaucoup. » car deirdre mccloskey – cas rare, ...","[0, 1]"


In [23]:
train_size = 0.8
train_dataset=df.sample(frac=train_size,random_state=200)
test_dataset=df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (2606, 2)
TRAIN Dataset: (2085, 2)
TEST Dataset: (521, 2)


In [24]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [25]:
import torch

class BERTClass(torch.nn.Module):
    def __init__(self):
      super(BERTClass, self).__init__()
      self.l1 = CamembertModel.from_pretrained("camembert-base")
      self.l3 = torch.nn.Linear(768, 2) #2 = binary classification
    
    def forward(self, ids, mask, token_type_ids):
      output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
      output = self.l3(output_1['pooler_output'])

      return output

model = BERTClass()

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [26]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [27]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids']
        mask = data['mask']
        token_type_ids = data['token_type_ids']
        targets = data['targets']
        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%100==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [28]:
for epoch in range(EPOCHS):
    train(epoch)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Epoch: 0, Loss:  0.7077392339706421
Epoch: 0, Loss:  0.387113094329834
Epoch: 0, Loss:  0.9573606848716736
Epoch: 0, Loss:  0.30780041217803955
Epoch: 0, Loss:  0.16869540512561798
Epoch: 0, Loss:  1.0870883464813232
Epoch: 0, Loss:  0.24561503529548645
Epoch: 0, Loss:  0.14414772391319275
Epoch: 0, Loss:  0.11035040020942688
Epoch: 0, Loss:  0.08362103998661041
Epoch: 0, Loss:  0.07819303870201111
Epoch: 0, Loss:  0.06055998057126999
Epoch: 0, Loss:  0.13416486978530884
Epoch: 0, Loss:  0.05172160267829895
Epoch: 0, Loss:  0.04834621399641037
Epoch: 0, Loss:  0.03872811049222946
Epoch: 0, Loss:  0.1030985489487648
Epoch: 0, Loss:  0.032506063580513
Epoch: 0, Loss:  0.07225216925144196
Epoch: 0, Loss:  0.0317205972969532
Epoch: 0, Loss:  0.35854795575141907


In [ ]:
# Pour enregistrer les stats a chaque epoque
training_stats = []
 
# Boucle d'entrainement
for epoch in range(0, epochs):
     
    print("")
    print(f'########## Epoch {epoch+1} / {epochs} ##########')
    print('Training...')
 
 
    # On initialise la loss pour cette epoque
    total_train_loss = 0
 
    # On met le modele en mode 'training'
    # Dans ce mode certaines couches du modele agissent differement
    model.train()
 
    # Pour chaque batch
    for step, batch in enumerate(train_dataloader):
 
        # On fait un print chaque 40 batchs
        if step % 40 == 0 and not step == 0:
            print(f'  Batch {step}  of {len(train_dataloader)}.')
         
        # On recupere les donnees du batch
        input_id = batch[0]
        attention_mask = batch[1]
        sentiment = batch[2]
 
        # On met le gradient a 0
        model.zero_grad()        
 
        # On passe la donnee au model et on recupere la loss et le logits (sortie avant fonction d'activation)
        loss, logits = model(input_id, 
                             token_type_ids=None, 
                             attention_mask=attention_mask, 
                             labels=sentiment)
 
        # On incremente la loss totale
        # .item() donne la valeur numerique de la loss
        total_train_loss += loss.item()
 
        # Backpropagtion
        loss.backward()
 
        # On actualise les parametrer grace a l'optimizer
        optimizer.step()
 
    # On calcule la  loss moyenne sur toute l'epoque
    avg_train_loss = total_train_loss / len(train_dataloader)   
 
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))  
     
    # Enregistrement des stats de l'epoque
    training_stats.append(
        {
            'epoch': epoch + 1,
            'Training Loss': avg_train_loss,
        }
    )
 
print("Model saved!")
torch.save(model.state_dict(), "./sentiments.pt")